In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "CBIR"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.extractNeuralFeatures import extract_features
from utils.extractNeuralFeatures import extrac_features_from_unlabled_dataset

In [4]:
chosen_net = networks.ALEXNET
linear_layers_to_remove = 3
X_train_ale_t, X_train_ale_n, y_train, X_test_ale_t, X_test_ale_n, y_test, fine_tune_layers = extract_features(train_set=datasets.TRAINING_LABELED,
                                                                                            test_set=datasets.TEST,
                                                                                            network=chosen_net,
                                                                                            layers_to_remove=linear_layers_to_remove, 
                                                                                            cuda=cuda)
print("Original classification layers:{}".format(chosen_net.value[1].classifier[:]))
print("---------------------------------------------------------------------------------")
print("Classification layers to fine tune:{}".format(fine_tune_layers[:]))

---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Train: 5020
Feature vector shape of Train: (5020, 9216)
Label vector shape of Train: (5020,)
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in Test: 11994
Feature vector shape of Test: (11994, 9216)
Label vector shape of Test: (11994,)
---------------------------------------------------------------------------------
Original classification layers:Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
-------------------------------------

In [5]:
chosen_net = networks.ALEXNET
layers_to_remove = 3
X_unlabled_mobi_t, X_unlabled_mobi_n, fine_tune_layers = extrac_features_from_unlabled_dataset(dataset=datasets.TRAINING_UNLABELED,
                                                                                            network=chosen_net,
                                                                                            layers_to_remove=layers_to_remove)
print("Original classification layers:{}".format(chosen_net.value[1].classifier[:]))
print("Classification layers to fine tune:{}".format(fine_tune_layers[:]))

---------------------------------------------------------------------------------
Done feat extraction, total n° of istances in unlabled: 113455
Feature vector shape of unlabled: (113455, 9216)
Label vector shape of unlabled: (113455,)
---------------------------------------------------------------------------------
Original classification layers:Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)
Classification layers to fine tune:Sequential(
  (0): Linear(in_features=9216, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Linear(in_features=4096, out_features=1000, bias=True)
)


In [6]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_ale_n, y_train)

KNeighborsClassifier(n_neighbors=10)

In [23]:

# Function to predict with threshold and progress bar
def knn_predict_with_threshold(knn, X, early_stop_at, threshold_distance):
    predictions = []
    dists=[]
    i=0
    for sample in tqdm(X):        
        if i>early_stop_at: break
        distances, _ = knn.kneighbors([sample], n_neighbors=knn.n_neighbors)
        if np.min(distances) > threshold_distance:
            predictions.append('unknown')            
            dists.append(-1)
        else:
            prediction = knn.predict([sample])
            predictions.append(prediction[0])
            dists.append(np.min(distances))
        
        i+=1
    return dists, predictions


dists, predictions = knn_predict_with_threshold(knn, X_unlabled_mobi_n, 30, threshold_distance=999)
print(np.mean(dists))
print(predictions)


  0%|          | 30/113455 [00:07<7:23:24,  4.26it/s]

93.66453768412272
[5, 49, 214, 0, 214, 214, 121, 0, 238, 121, 1, 21, 167, 0, 9, 0, 49, 8, 8, 0, 0, 0, 1, 124, 4, 214, 121, 0, 120, 188]
